### Evaluation Dataset Link: https://drive.google.com/file/d/1NaDz3Y4j01GrnJRY2ln2DN9gfG9QgSl_/view?usp=drive_link

In [ ]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 39.4 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 46.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 29.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import scipy.linalg
from scipy.spatial.distance import cosine
import math
from PIL import Image
import os
from tqdm import tqdm
import random
import pandas as pd

# Set the random seed for reproducibility
def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Define paths for local machine
REAL_MONET_PATH = "Eval"  # Update with actual path for real Monet images
GENERATED_MONET_PATH = "submission"  # Update with actual path for generated Monet images
batch_size = 64

# Set a random seed for reproducibility
set_random_seed(42)

In [ ]:
# Helper function to load images
def load_images(image_paths, transform):
    images = []
    for path in image_paths:
        image = Image.open(path).convert('RGB')

        # Ensure images are 256x256
        if image.size != (256, 256):
            raise ValueError(f"Image {path} is not 256x256 in size.")

        # Ensure images are in .jpg format
        if not path.lower().endswith('.jpg'):
            raise ValueError(f"Image {path} is not in .jpg format.")

        image = transform(image)
        images.append(image)

    return torch.stack(images)

# Helper function to calculate activations using InceptionV3
def calculate_activations(images, model, device):
    images = images.to(device)
    with torch.no_grad():
        features = model(images)
    return features.cpu().numpy()

In [ ]:
# Function to subsample the larger dataset (real or generated)
def subsample_datasets(real_image_paths, generated_image_paths):
    # Determine the smaller and larger dataset
    real_count = len(real_image_paths)
    generated_count = len(generated_image_paths)

    if real_count > generated_count:
        # Subsample the real images to match the number of generated images
        print(f"Subsampling real images from {real_count} to {generated_count}")
        real_image_paths = random.sample(real_image_paths, generated_count)
    elif generated_count > real_count:
        # Subsample the generated images to match the number of real images
        print(f"Subsampling generated images from {generated_count} to {real_count}")
        generated_image_paths = random.sample(generated_image_paths, real_count)

    return real_image_paths, generated_image_paths

In [ ]:
# Function to compute FID and MiFID
def calculate_mifid(real_image_paths, generated_image_paths, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load InceptionV3 model
    inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
    inception.fc = nn.Identity()  # Remove classification layer
    inception.eval()
    inception = inception.to(device)

    # Define the image transformation
    transform = transforms.Compose([
        transforms.Resize(int(256 * 1.33)),  # Resize with scaling
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),  # ImageNet stats
    ])

    # Subsample datasets if needed
    real_image_paths, generated_image_paths = subsample_datasets(real_image_paths, generated_image_paths)

    # Process real images in batches
    real_activations = []
    for i in tqdm(range(0, len(real_image_paths), batch_size), desc="Processing Real Images"):
        batch_paths = real_image_paths[i:i+batch_size]
        batch_images = load_images(batch_paths, transform)
        real_activations.append(calculate_activations(batch_images, inception, device))
    real_activations = np.concatenate(real_activations, axis=0)

    # Process generated images in batches
    generated_activations = []
    for i in tqdm(range(0, len(generated_image_paths), batch_size), desc="Processing Generated Images"):
        batch_paths = generated_image_paths[i:i+batch_size]
        batch_images = load_images(batch_paths, transform)
        generated_activations.append(calculate_activations(batch_images, inception, device))
    generated_activations = np.concatenate(generated_activations, axis=0)

    # Calculate mean and covariance
    real_mu = np.mean(real_activations, axis=0)
    generated_mu = np.mean(generated_activations, axis=0)
    real_sigma = np.cov(real_activations, rowvar=False)
    generated_sigma = np.cov(generated_activations, rowvar=False)

    # Compute FID
    diff = real_mu - generated_mu
    covmean, _ = scipy.linalg.sqrtm(real_sigma.dot(generated_sigma), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(real_sigma + generated_sigma - 2 * covmean)

    # Calculate MiFID
    # Calculate the memorization distance between each generated image and the real images
    memorization_distances = [
        min([cosine(g_feat, r_feat) for r_feat in real_activations])
        for g_feat in generated_activations
    ]
    # Calculate the MiFID as the mean memorization distance
    mifid = np.mean(memorization_distances)

    print(f"MiFID Score: {mifid}")
    return fid, mifid

In [ ]:
# Main function for evaluation
def evaluate():
    # Get list of real and generated images
    real_image_paths = sorted([os.path.join(REAL_MONET_PATH, f) for f in os.listdir(REAL_MONET_PATH) if f.endswith('.jpg')])
    generated_image_paths = sorted([os.path.join(GENERATED_MONET_PATH, f) for f in os.listdir(GENERATED_MONET_PATH) if f.endswith('.jpg')])

    # Check that there are more than 300 generated images
    if len(generated_image_paths) < 300:
        raise ValueError("The number of generated images should be greater than 300.")

    # Compute FID and MiFID scores
    fid_score, mifid_score = calculate_mifid(real_image_paths, generated_image_paths, batch_size=batch_size)

    # Save results to CSV
    result_df = pd.DataFrame({
        'ID': [1],
        'FID': [fid_score],
        'MiFID': [mifid_score]
    })

    result_df.to_csv('submission.csv', index=False)
    print(result_df)
    print("Results saved to 'submission.csv'")

# Run the evaluation
if __name__ == "__main__":
    evaluate()

Subsampling generated images from 7038 to 933


Processing Generated Images: 100%|██████████| 15/15 [00:04<00:00,  3.21it/s]


MiFID Score: 0.2208790270724342
   ID        FID     MiFID
0   1  56.999961  0.220879
Results saved to 'submission.csv'
